# Configuração Inicial

## Importando depedências

In [ ]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

## Configuração inicial do DataFrame

In [ ]:
df = pd.read_csv('../../datasets/dataset-csv/avaliacao_dataset.csv')

In [ ]:
df.head()

### alterando os valores da coluna avaliação de acordo com esse template:
- 1: -2,
- 2: -1,
- 3: 1,
- 4: 1,
- 5: 2

Com isso mudamos nosso modelo de avaliação de um sistema com 5 tipo para um com 4, retirando o valor "0" que normalmente os modelos de recomendação tem dificultade de utiliza-los.

In [ ]:
mapa_avaliacoes = {
    1: -2,
    2: -1,
    3: 1,
    4: 1,
    5: 2
}

df['avaliação'] = df['avaliação'].map(mapa_avaliacoes)

In [ ]:
df.head()

In [ ]:
df.avaliação.value_counts()

# iniciando o modelo

In [ ]:
reader = Reader(rating_scale=(-2, 2))
data = Dataset.load_from_df(df, reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.25)

model = SVD()
model.fit(trainset)
predictions = model.test(testset)

## Métricas de avaliação

In [ ]:
from surprise import accuracy
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import label_binarize

In [ ]:
def plot_metrics(metrics):
    metric_names = list(metrics.keys())
    metric_values = list(metrics.values())

    # Filtrar métricas que são valores numéricos
    metric_names_numeric = [name for name, value in zip(metric_names, metric_values) if isinstance(value, (int, float))]
    metric_values_numeric = [value for value in metric_values if isinstance(value, (int, float))]

    plt.figure(figsize=(10, 6))
    plt.bar(metric_names_numeric, metric_values_numeric, color='skyblue')
    plt.xlabel('Metrics')
    plt.ylabel('Values')
    plt.title('Evaluation Metrics')
    plt.ylim(0, max(metric_values_numeric) * 1.2)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    for i, value in enumerate(metric_values_numeric):
        plt.text(i, value + 0.01, f"{value:.2f}", ha='center', va='bottom')

    plt.show()

In [ ]:
def evaluate_predictions(predictions):
    # Converter as previsões em listas de valores reais e preditos
    true_ratings = [pred.r_ui for pred in predictions]
    est_ratings = [pred.est for pred in predictions]

    # Calcular RMSE e MAE
    rmse = accuracy.rmse(predictions, verbose=False)
    mae = accuracy.mae(predictions, verbose=False)

    est_transformed = [
        min(2, round(r + 0.5)) if r > 0 else max(-2, round(r - 0.5))
        for r in est_ratings
    ]

    # Calcular Precision, Recall e F1-Score
    precision = precision_score(true_ratings, est_transformed, average="macro")
    recall = recall_score(true_ratings, est_transformed, average="macro")
    f1 = f1_score(true_ratings, est_transformed, average="macro")

    # Classes únicas
    classes = np.array([-2, -1, 1, 2])

    # Binarizar os ratings verdadeiros para o cálculo de AUC-ROC
    true_ratings_binarized = label_binarize(true_ratings, classes=classes)

    est_probabilities = np.random.rand(len(est_ratings), len(classes))
    est_probabilities /= est_probabilities.sum(axis=1)[:, np.newaxis]

    # Calcular AUC-ROC, considerando o problema multiclasse
    if len(np.unique(true_ratings)) > 1:  # Verificar se há mais de uma classe em y_true
        auc_roc = roc_auc_score(true_ratings_binarized, est_probabilities, multi_class='ovr', average='macro')
    else:
        auc_roc = "n/a"  # Não aplicável se não houver ambas as classes

    metrics = {
        "RMSE": rmse,
        "MAE": mae,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "AUC-ROC": auc_roc,
    }

    plot_metrics(metrics)
    return metrics

In [ ]:
evaluate_predictions(predictions)

As métricas RMSE(Raiz do Erro Quadrático Médio) e MAE(Erro Absoluto Médio) mostram como o modelo está errando suas predições quando comparado com os valores reais do dataset `testset`. Ou seja como o RMSE e o MAE estão próximos os erros não estão sendo muito váriantes de 1 para cima ou para baixo.

Explicação das outras métricas:

1. Precision (Precisão)
A precisão é a proporção de verdadeiros positivos em relação ao total de previsões positivas.

$$
\text{Precision} = \frac{TP}{TP + FP}
$$

Valor: **0.727**

2. Recall (Revocação)
A revocação é a proporção de verdadeiros positivos em relação ao total de instâncias reais positivas.

$$
\text{Recall} = \frac{TP}{TP + FN}
$$

Valor: **0.464**

3. F1-Score
O F1-Score é a média harmônica entre precisão e revocação.

$$
\text{F1-Score} = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}
$$

Valor: **0.425**

4. AUC-ROC (Área Sob a Curva - Característica de Operação do Receptor)
A AUC-ROC mede a capacidade do modelo de distinguir entre classes.

Valor: **0.461**

## Função de recomendação para um usuário

In [ ]:
def predictRecomendationModel(id_proponente, model=model):
    all_projects = set(df['id_projeto'])  # Todos os projetos
    rated_projects = set(df[df['id_proponente'] == id_proponente]['id_projeto'])  # Projetos já avaliados pelo usuário
    projects_to_predict = all_projects - rated_projects # Projetos não avaliados

    predictions = []
    for project_id in projects_to_predict:
        predictions.append((project_id, model.predict(id_proponente, project_id).est))
    
    # Ordenar as previsões por estimativa de maior para menor
    predictions.sort(key=lambda x: x[1], reverse=True)
    # Exibir as melhores recomendações
    top_recommendations = predictions[:10]  # Top 10 recomendações, podendo mudar para aumentar o número de recomendações
    print(top_recommendations)

Cada recomendação acompanha o id_projeto como primeiro elemento e a nota atribuída pelo modelo de recomendação como segundo elemento

In [ ]:
predictRecomendationModel(1)

# Salvando o modelo com joblib

In [ ]:
from joblib import dump, load

# Suponha que 'model' é o seu modelo SVD treinado
dump(model, './saveModel/svd_model.joblib')